In [1]:
import ujson
import re
import pandas as pd

from glob import glob
from gensim.models.word2vec import Word2Vec
from itertools import product
from IPython.display import Markdown, display
from wordfreq import top_n_list
from tqdm import tqdm_notebook
from tabulate import tabulate

In [2]:
pd.options.display.max_rows = 1000

In [3]:
def read_tweets(pattern, fcount=None):
    
    paths = glob(pattern)
    
    if fcount:
        paths = paths[:fcount]
    
    tweets = []
    for path in paths:
        with open(path) as fh:
            for line in fh:
                
                tweet = ujson.loads(line)
                text = re.sub('(#|@|http)\S+', '', tweet['body'])
                tokens = re.findall('[a-z0-9]+', text.lower())
                tweets.append(tokens)
                
    return tweets

In [4]:
rural = read_tweets('../../data/geo-lt10k.json/*.json', 60)

In [5]:
urban = read_tweets('../../data/geo-gt1m.json/*.json', 20)

In [6]:
len(rural)

11780936

In [7]:
len(urban)

12325982

In [108]:
rural_model = Word2Vec(rural, min_count=1000, workers=8)

In [109]:
urban_model = Word2Vec(urban, min_count=1000, workers=8)

In [11]:
rural_model.save('../data/models/rural.bin')

In [12]:
urban_model.save('../data/models/urban.bin')

In [110]:
rural_model.most_similar('earth')

[('planet', 0.7846516370773315),
 ('universe', 0.495510071516037),
 ('purpose', 0.4822278320789337),
 ('moon', 0.4742686450481415),
 ('world', 0.4736853539943695),
 ('alien', 0.47179871797561646),
 ('species', 0.46059975028038025),
 ('mars', 0.4551383852958679),
 ('nature', 0.44583749771118164),
 ('shining', 0.4440971910953522)]

In [111]:
urban_model.most_similar('earth')

[('planet', 0.7880303859710693),
 ('darkness', 0.49556586146354675),
 ('nature', 0.48827022314071655),
 ('thrive', 0.4876272678375244),
 ('invisible', 0.48423391580581665),
 ('heaven', 0.4732300639152527),
 ('moon', 0.4650537371635437),
 ('trees', 0.45670124888420105),
 ('species', 0.45475828647613525),
 ('christ', 0.451892614364624)]

In [112]:
def score_shift(m1, m2, token, n=10):
    
    m1_similar = [t for t, _ in m1.most_similar(token, topn=n)]
    m2_similar = [t for t, _ in m2.most_similar(token, topn=n)]
    
    score = 0
    for t1, t2 in product(m1_similar, m2_similar):
        if t1 in m2 and t2 in m2:
            score += m2.similarity(t1, t2)
        
    return score

In [121]:
rural_vocab = set([t for t, v in rural_model.wv.vocab.items() if v.count > 5000])
urban_vocab = set([t for t, v in urban_model.wv.vocab.items() if v.count > 5000])

In [122]:
len(rural_vocab)

2049

In [123]:
len(urban_vocab)

2268

In [124]:
vocab = set.intersection(rural_vocab, urban_vocab)

In [125]:
len(vocab)

1897

In [126]:
data = []

for token in tqdm_notebook(vocab):
    try:
        score = score_shift(urban_model, rural_model, token)
        data.append((token, score))
    except: pass

HBox(children=(IntProgress(value=0, max=1897), HTML(value='')))

In [127]:
df = pd.DataFrame(data, columns=('token', 'score'))

In [131]:
df.sort_values('score').head(300)

,token,score
868,lady,1.000451
296,lo,1.521036
1241,fair,2.445868
784,mostly,4.540545
1822,race,4.615573
848,track,5.253162
993,network,5.509127
843,gaga,6.023442
1847,lt,7.210392
1564,yasss,9.599905


In [132]:
def compare(token):
    
    u_similar = urban_model.most_similar(token)
    r_similar = rural_model.most_similar(token)
    
    u_tokens = [[t] for t, _ in u_similar]
    r_tokens = [[t] for t, _ in r_similar]
    
    return u_tokens, r_tokens

In [168]:
compare('selfish')

([['petty'],
  ['miserable'],
  ['rude'],
  ['bitter'],
  ['disrespectful'],
  ['childish'],
  ['ignorant'],
  ['delusional'],
  ['stupid'],
  ['reckless']],
 [['heartless'],
  ['rude'],
  ['childish'],
  ['disrespectful'],
  ['petty'],
  ['miserable'],
  ['immature'],
  ['spoiled'],
  ['cruel'],
  ['jealous']])

In [133]:
for token in df.sort_values('score').head(300)['token']:
    u, r = compare(token)
    display(Markdown(f'# {token}'))
    display(Markdown(f'#### Urban'))
    print(tabulate(u))
    display(Markdown(f'#### Rural'))
    print(tabulate(r))

# lady

#### Urban

--------
girl
woman
guy
gal
man
boy
chick
kid
bride
coworker
--------


#### Rural

--------
knights
bulldogs
tigers
timeout
panthers
wildcats
halftime
lions
boys
indians
--------


...

# doubt

#### Urban

----------
worries
slander
think
regrets
prob
clue
sadly
promise
disrespect
trust
----------


#### Rural

-------
worries
fear
regrets
trust
sadly
prob
respect
problem
matter
lie
-------


- lady - person vs sports team
- stranger - person vs adjective

- sessions **
- russian **
- bush **

- genuine - sincere vs. gemstone
- track - song vs running

- raw

- march ***
- rob - verb vs name
- race ***
- drama

- associate - verb vs noun
- noble - verb vs name

- foundation - makeup vs ?
- waters - outdoors vs politics
- faced ?

- single - song vs baseball
- boys - boys vs sports team
- apple - fruit vs company
- clear - obvious vs weather
- moderate - politics vs weather

- saint ***
- he - obama, putin, postus
- sacred ???
- holywood

- catholic ***
- church ***

- black